In [20]:
import qsharp
import qsharp_widgets
import json
from diskcache import Cache
import re_utils
from math import ceil

re_utils.DEBUG = True

cache = Cache("~/quant-arith-cache/re-multipliers")
qsharp.init(project_root="../")

@cache.memoize()
def estimate_resources_out_of_place_adder_pareto(op, n):
    if op == "QuantumArithmetic.WBC2023.AddWithOp":
        radix = ceil(n / 2)
        est = qsharp.estimate(f"EstimateUtils.RunRadix({n},{radix},{op},Std.Arithmetic.RippleCarryCGAddLE)", params={"estimateType": "frontier"})
    else:
        est = qsharp.estimate(f"EstimateUtils.Run3WayOp({n},{n},{n},{op})", params={"estimateType": "frontier"})
    return est
  
# These all are out-of-place adders modulo 2^n.
ops_and_max_n = [
  ("Std.Arithmetic.RippleCarryCGAddLE", "Gidney", 2**20),
  ("Std.Arithmetic.LookAheadDKRSAddLE", "DKRS", 2**20),
  ("QuantumArithmetic.CT2002.Add", "CT", 2**20),
  ("QuantumArithmetic.GKDKH2021.Add_Mod2N", "Gayathri", 2**20),
  ("QuantumArithmetic.WLLQW2016.Add_Mod2N", "Wang", 2**20),      
  ("QuantumArithmetic.WBC2023.AddWithOp", "Higher Radix", 2**20),
  ("QuantumArithmetic.SC2023.Add_Mod2N", "Ling Structure", 110218),
]

In [23]:
n = 2**14
results = {} 
i = 0 
for op, _, _ in ops_and_max_n:
    results[i] = estimate_resources_out_of_place_adder_pareto(op, n)
    i += 1

qsharp_widgets.EstimatesOverview(results, runNames=[name for _, name, _ in ops_and_max_n])

EstimatesOverview(estimates={0: {'status': 'success', 'jobParams': {'qecScheme': {'name': 'surface_code', 'err…

The plot above shows two main groups of the adders. In order to show more of the differences, the out of place adders were run again (without the Ling Structure adder) with `n=2**3` to show the differences. Even with the smaller input, Gidney, Gayathri, and Wang all shared the same resources.

In [19]:
n = 2**3
n_filter = 2**20
results = {} 
i = 0 
for op, _, max_n in ops_and_max_n:
    if max_n >= n_filter:
        results[i] = estimate_resources_out_of_place_adder_pareto(op, n)
        i += 1

qsharp_widgets.EstimatesOverview(results, runNames=[name for _, name, max_n in ops_and_max_n if max_n >= n_filter])

EstimatesOverview(estimates={0: {'status': 'success', 'jobParams': {'qecScheme': {'name': 'surface_code', 'err…